
## Detect which clusters are using legacy DBR versions. 
### Instructions: import and run this notebook

In [0]:
CREATE OR REPLACE TEMP VIEW legacy_dbrs AS 
WITH clusters_dbr_versions AS (
  SELECT DISTINCT
    account_id,
    workspace_id,
    cluster_id,
    cluster_name,
    owned_by,
    dbr_version,
    TRY_CAST(split_part(dbr_version, '.', 1) AS INT) AS major_version,
    TRY_CAST(split_part(dbr_version, '.', 2) AS INT) AS minor_version,
    CONCAT(split_part(dbr_version, '.', 1), '.', split_part(dbr_version, '.', 2)) AS dbr_version
  FROM
    system.compute.clusters
),
usage AS (
  SELECT
    account_id,
    workspace_id,
    usage_metadata.cluster_id,
    usage_unit,
    usage_quantity
  FROM
    system.billing.usage
  WHERE
    usage_date > CURRENT_DATE() - INTERVAL 90 DAYS
),
workspace_info AS (
  SELECT
    account_id,
    workspace_id,
    workspace_name,
    workspace_url
  FROM
    system.access.workspaces_latest
)
SELECT
  cdv.account_id,
  cdv.workspace_id,
  wi.workspace_name,
  wi.workspace_url,
  cdv.cluster_id,
  cdv.dbr_version,
  usage_unit,
  ROUND(SUM(usage_quantity), 2) AS total_usage_dbu
FROM
  clusters_dbr_versions cdv
    INNER JOIN usage u
      ON cdv.cluster_id = u.cluster_id
      AND cdv.account_id = u.account_id
      AND cdv.workspace_id = u.workspace_id
    LEFT JOIN workspace_info wi
      ON cdv.workspace_id = wi.workspace_id
      AND cdv.account_id = wi.account_id
WHERE
  dbr_version <= "10.4"
GROUP BY 1,2,3,4,5,6,7
ORDER BY
  workspace_id

In [0]:
SELECT * FROM legacy_dbrs

In [0]:
SELECT 
  dbr_version, 
  workspace_id, 
  COUNT(DISTINCT cluster_id) total_clusters, 
  SUM(total_usage_dbu)  AS total_usage_dbu
FROM legacy_dbrs
GROUP BY dbr_version, workspace_id
ORDER BY dbr_version, workspace_id